<a href="https://colab.research.google.com/github/Atsumu-Hiranishi/Chapter2/blob/main/Zettai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Zettai.kt

1. クラス構成


```
data class Zettai(val lists: Map<User, List<ToDoList>>) : HttpHandler
```


- Zettaiクラス: HttpHandlerインターフェースを実装するデータクラスで、アプリケーションのエンドポイントを定義している。
- プロパティ lists: 各ユーザーのToDoListを保持するマップで、Userがキー、List\<ToDoList>が値になっている。このマップから、ユーザーごとのリストを取り出して応答する。

2. ルーティング設定


```
val routes = routes(
    "/todo/{user}/{list}" bind Method.GET to ::getToDoList
)
```


- /todo/\{user}/\{list}というURLパターンに対してGETメソッドをバインドし、getToDoList関数を呼び出す。
- userとlistの部分がURLパスパラメータとして扱われ、特定のユーザーとリスト名に基づいたリクエストに応答する。

3. invoke 関数


```
override fun invoke(request: Request): Response = routes(request)
```


- HttpHandlerインターフェースのinvokeメソッドをオーバーライドし、リクエストを受け取って登録されたroutesに処理を委譲している。
- これにより、HTTPリクエストがZettaiクラスに届いたときに適切なルートで処理が行われる。

4. getToDoList 関数


```
private fun getToDoList(request: Request): Response =
    request.let(::extractListData)
        .let(::fetchListContent)
        .let(::renderHtml)
        .let(::createResponse)
```


- extractListData: リクエストからユーザーとリスト名を抽出し、それぞれUserとListNameのインスタンスとしてペアで返す。
- fetchListContent: listsマップから、UserとListNameに対応するToDoListを取得する。
- renderHtml: 指定したToDoListをHTML形式にレンダリングする。
- createResponse: HTMLページをレスポンスの本文として設定し、レスポンスを返す。

5. extractListData 関数


```
fun extractListData(request: Request): Pair<User, ListName> {
    val user = request.path("user") ?: error("User missing")
    val list = request.path("list") ?: error("List missing")
    return User(user) to ListName(list)
}
```


- リクエストからuserとlistのパスパラメータを取得し、取得できない場合はエラーメッセージを投げる。
- `user`と`list`を`User`および`ListName`として返す。

6. fetchListContent 関数


```
fun fetchListContent(listId: Pair<User, ListName>): ToDoList =
    lists[listId.first]
        ?.firstOrNull { it.listName == listId.second }
        ?: error("List unknown")
```


- 指定したUserに対応するToDoリストを検索し、ListNameが一致する最初のToDoListを取得する。
- 対応するリストが見つからない場合はエラーを返す。

7. createResponse 関数


```
fun createResponse(html: HtmlPage): Response = Response(Status.OK).body(html.raw)
```


- html引数: HtmlPage型で、renderHtmlによって生成されたHTMLページ。
- Responseオブジェクトを返し、ステータスコード200とHTMLの本文を設定する。
